<a href="https://colab.research.google.com/github/ishwarvenugopal/youtube-weeklyRoundup/blob/main/YouTube_WeeklyRoundup_MalayalamNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import re
import datetime
from json import JSONDecoder
import string

In [2]:
no_of_days = 7

https://stackoverflow.com/questions/54235528/how-to-find-json-object-in-text-with-python

In [3]:
def extract_json_objects(text, decoder=JSONDecoder()):
    """Find JSON objects in text, and yield the decoded JSON data

    Does not attempt to look for JSON arrays, text, or other JSON types outside
    of a parent JSON object.

    """
    pos = 0
    while True:
        match = text.find('{', pos)
        if match == -1:
            break
        try:
            result, index = decoder.raw_decode(text[match:])
            yield result
            pos = match + index
        except ValueError:
            pos = match + 1

In [4]:
today = datetime.datetime.now().date()
req_dates = []
for i in range(no_of_days):
  req_date = today - datetime.timedelta(days = i)
  month = req_date.strftime("%B")
  day = req_date.day
  year = req_date.year
  req_string = '{} {}, {}'.format(month,day,year)
  req_dates.append(req_string)

In [5]:
req_dates.reverse()

In [6]:
req_dates

['October 24, 2022',
 'October 25, 2022',
 'October 26, 2022',
 'October 27, 2022',
 'October 28, 2022',
 'October 29, 2022',
 'October 30, 2022']

In [7]:
def get_youtubeLink(video):
  videoTitle = video['videoRenderer']["title"]["runs"][0]["text"]
  videoId = video['videoRenderer']['videoId']
  url = "{} https://www.youtube.com/watch?v={}".format(videoTitle,videoId)
  return url, videoId, videoTitle

In [8]:
req_VideoIds = []
for req_date in req_dates:
  date_noPunct = ''.join([x for x in req_date if x not in string.punctuation and x!=' '])
  search_keyword='malayalam news headlines "{}"'.format(req_date)
  search_url = "https://www.youtube.com/results?search_query=" + search_keyword
  response = requests.get(search_url)
  contents = response.content.decode('utf-8')
  for result in extract_json_objects(contents):
    if 'responseContext' in result.keys():
      req_dict = result
  key = list(req_dict.keys())[2]
  req_list = req_dict[key]['twoColumnSearchResultsRenderer']['primaryContents']['sectionListRenderer']['contents'][0]
  videoList = req_list['itemSectionRenderer']['contents']
  req_videos = {
    '10PMNews': [],
    '9PMNews': [],
    '630PMNews': [],
    '6PMNews': [],
    '130PMNews': [],
    '1PMNews': [],
    '10AMNews': [],
    '9AMNews': [],
  }
  for video in videoList:
    present = req_date in json.dumps(video)
    if present:
      if 'videoRenderer' in video.keys():
        url, videoId, videoTitle = get_youtubeLink(video)
        if 'News' in videoTitle:
          titleSplit = videoTitle.split('|')
          titleDate = ''.join([x for x in titleSplit[-1] if x not in string.punctuation and x!=' '])
          titleNewsTime = ''.join([x for x in titleSplit[-2] if x not in string.punctuation and x!=' '])
          if date_noPunct in titleDate:
            if titleNewsTime in req_videos.keys():
              req_videos[titleNewsTime].append([url, videoId])
  for key, value in req_videos.items():
    if len(value)>0:
      print(value[0][0])
      req_VideoIds.append(value[0][1])
      break

പത്തു മണി വാർത്ത | 10 PM News | October 24, 2022 https://www.youtube.com/watch?v=5hQsdSAOpIU
പത്തു മണി വാർത്ത | 10 P M News | October 25, 2022 https://www.youtube.com/watch?v=eE4PtDRLhOI
പത്തു മണി വാർത്ത | 10 P M News | October 26, 2022 https://www.youtube.com/watch?v=-OGMg_hfDnY
പത്തു മണി വാർത്ത | 10 P M News | October 27, 2022 https://www.youtube.com/watch?v=uQwLe3eXWys
പത്തു മണി വാർത്ത | 10 P M News | October 28, 2022 https://www.youtube.com/watch?v=IX4U7p0BNW8
പത്തു മണി വാർത്ത | 10 P M News | October 29, 2022 https://www.youtube.com/watch?v=TQI6N1VzTuk
ഒൻപതു മണി വാർത്ത | 9 AM News | October 30, 2022 https://www.youtube.com/watch?v=7to1eX4S_4U


In [9]:
playlist_link = 'http://www.youtube.com/watch_videos?video_ids='
for videoID in req_VideoIds:
  playlist_link = playlist_link + '{},'.format(videoID)
print(playlist_link)

http://www.youtube.com/watch_videos?video_ids=5hQsdSAOpIU,eE4PtDRLhOI,-OGMg_hfDnY,uQwLe3eXWys,IX4U7p0BNW8,TQI6N1VzTuk,7to1eX4S_4U,
